this notebook was used to create a standartized inventory for pico, including the spiketimes and psth h5 files. additionally, the information at hand was extracted from the excel file and saved.

### Step 1: Load modules and functions

In [17]:
import numpy as np
import io
import pandas as pd
import os
import fnmatch
import shutil
import re
from ruamel.yaml import YAML
import scipy.io as sio

def find_directories_with_name(root_dir, directory_name):
    directory_paths = []
    for foldername, subfolders, _ in os.walk(root_dir):
        # print(foldername, subfolders, _)
        if directory_name in subfolders:
            added = True
            for subsubfolder in os.listdir(os.path.join(foldername, directory_name)):
                if os.path.isdir(os.path.join(foldername, directory_name, subsubfolder)):
                    directory_paths.append(os.path.join(foldername, directory_name, subsubfolder))
                else: added = False
            if added == False: directory_paths.append(os.path.join(foldername, directory_name))

    return directory_paths

def find_files_with_extension(root_dir, extension):
    file_paths = []
    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in fnmatch.filter(filenames, f'*.{extension}'):
            file_paths.append(os.path.join(foldername, filename))
    return file_paths

def get_date(date):
    """
    Get the Dates from the Excel file.
    """

    try:
        dates = date.split('/')
        if len(dates[-1])>2:year = dates[-1][2:]
        else: year=f'{dates[-1]}'
        if len(dates[0])>1: month = dates[0]
        else: month = f'0{dates[0]}'
        if len(dates[1])>1:day=dates[1]
        else:day=f'0{dates[1]}'
        

    except:
        try:
            dates = str(date).split()[0].split('-')
            if len(dates)==3:
                year=dates[0][2:]
                month=dates[1]
                if len(dates[2])>1:day=dates[2]
                else:day=f'0{dates[2]}'
            else:
                pass
        except:
            pass

    try: return day, month, year
    except: return 0,0,0
  
def find_files_with_pattern(root_dir, pattern):
    """
    Find Associated .h5 File for each SpikeTime Folder.
    """
    return [os.path.join(foldername, filename) for foldername, subfolders, filenames in os.walk(root_dir) for filename in filenames if filename.startswith(pattern) and filename.endswith('.h5')]

def combine_channels(proc_dir, savedir, num_channels):

    filename = proc_dir.split('/')[-2]
    ch_files = os.listdir(proc_dir)
    ch_files.sort()
    if not len(ch_files) == num_channels:  # Skip if not all channel files present
        print('RERUN, not all channel files are present!')
    ch_files = [item for item in ch_files if not item.startswith('.')]
    psth = [sio.loadmat(os.path.join(proc_dir, f), squeeze_me=True, variable_names='psth')['psth'] for f in ch_files]
    psth = np.asarray(psth)  # channels x stimuli x reps x timebins
    psth = np.moveaxis(psth, 0, -1)  # stimuli x reps x timebins x channels
    #logging.debug(psth.shape)

    meta = sio.loadmat(os.path.join(proc_dir, ch_files[0]), squeeze_me=True, variable_names='meta')['meta']
    psth = {'psth': psth, 'meta': meta}
    sio.savemat(os.path.join(savedir, filename+'_psth.mat'), psth)

cwd = os.getcwd()

inventory_excel_file_name = 'pico_inventory.xlsx'


### Step 2: Find directories in Sarah's braintree

In [2]:
############### Find Directories of SpikeTimes ################################
###############################################################################
parent_directory = '/braintree/data2/active/users/sgouldin/projects'
target_directory_name = 'spikeTime'
spikeTime_directories = find_directories_with_name(parent_directory, target_directory_name)
print(len(spikeTime_directories))

spike_parent = []
for path in spikeTime_directories:
    try: 
        if path.split('/')[9] == 'pico':
            spike_parent.append(os.path.join(*path.split('/')[0:-1]))
    except: pass
    #spike_parent.append(os.path.join(*path.split('/')[0:10]))

############### Find Directories of psth ######################################
###############################################################################
psth_directories = find_directories_with_name(parent_directory, 'psth')
print(len(psth_directories))
psth_parent = []
for path in psth_directories:
    try: 
        if path.split('/')[9] == 'pico':
            psth_parent.append(os.path.join(*path.split('/')[0:-1]))
    except: pass

############### Find Directories of IntanRaw ##################################
###############################################################################
target_directory_name = 'intanraw'
intanRaw_directories = find_directories_with_name(parent_directory, target_directory_name)
print(len(intanRaw_directories))

intanRaw_parents = []
for path in intanRaw_directories:
    try: 
        if path.split('/')[9] == 'pico':
            intanRaw_parents.append(os.path.join(*path.split('/')[10]))
    except: pass #print(path)
    # intanRaw_parents.append(os.path.join(*path.split('/')[10]))


############### Find Directories for PSTH h5 Files ############################
###############################################################################
extension = 'h5'
h5_directories = find_files_with_extension(parent_directory, extension)
print(len(h5_directories))


In [3]:
set(spike_parent) - set(psth_parent)

{'braintree/data2/active/users/sgouldin/projects/NSD-COCO/monkeys/pico/intanproc/pico_NSD-COCO_230313_133413',
 'braintree/data2/active/users/sgouldin/projects/NSD-COCO/monkeys/pico/intanproc/pico_NSD-COCO_230314_111731',
 'braintree/data2/active/users/sgouldin/projects/NSD-COCO/monkeys/pico/intanproc/pico_NSD-COCO_230315_120952',
 'braintree/data2/active/users/sgouldin/projects/gestalt/monkeys/pico/intanproc/pico_gestalt_230503_132701',
 'braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/intanproc/pico_normalizers_220705_143816',
 'braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/intanproc/pico_normalizers_220706_142235',
 'braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/intanproc/pico_normalizers_230804_152622',
 'braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/intanproc/pico_normalizers_230808_132257',
 'braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/intanproc/pico_n

In [4]:
############### Get Info for Pico Data ########################################
###############################################################################

intan_names = []
names = []
dates = []
times = []
pico_intanRaw_directories  = []

for path in intanRaw_directories:
    try:
        if path.split('/')[9] == 'pico': 
            try: 
                intan_names.append(path.split('/')[11])
                dates.append((path.split('/')[11].split('_')[-2]))
                times.append((path.split('/')[11].split('_')[-1]))
                names.append(path.split('/')[7])
                pico_intanRaw_directories.append(path)
            except: pass
    except: pass

spike_names = []
pico_spikeTime_directories = []
for path in spikeTime_directories:
    try:
        if path.split('/')[9] == 'pico':
            spike_names.append(path.split('/')[11])
            pico_spikeTime_directories.append(path)
    except: pass #print(path)


psth_names = []
pico_psth_directories = []
for path in psth_directories:
    try:
        if path.split('/')[9] == 'pico':
            # for ele in os.listdir(os.path.join('/', *path.split('/')[0:-1])):
            #     if ele.endswith(".mat"): 
            #         psth_names.append(path.split('/')[11])
            #         path_ = os.path.join(os.path.join('/', *path.split('/')[0:-1], ele))
            pico_psth_directories.append(path)
            psth_names.append(path.split('/')[11])
    except: pass #print(path)


h5_names = []
h5_dates = []
pico_h5_directories = []
for path in h5_directories:
    try:
        if path.split('/')[9] == 'pico':
            h5_names.append(path.split('/')[7])
            h5_dates.append(path.split('/')[11].split('.')[0])
            pico_h5_directories.append(path)
    except: pass #print(path)

### Step 3: Write inventory excel file

In [6]:
############### Create DataFrame ##############################################
###############################################################################
df = pd.DataFrame({'ImageSet': names})
df['date'] = dates
df['time'] = times
df['Has SpikeTime']     = [0]*len(pico_intanRaw_directories)
df['Has psth']     = [0]*len(pico_intanRaw_directories)
df['Has h5']            = [0]*len(pico_intanRaw_directories)
df['Has Excel']         = [0]*len(pico_intanRaw_directories)
df['(excel) Index']          = ['']*len(pico_intanRaw_directories)
df['(excel) Stimuli']          = ['']*len(pico_intanRaw_directories)
df['(excel) Num Reps']          = ['']*len(pico_intanRaw_directories)
df['(excel) Num Images']        = ['']*len(pico_intanRaw_directories)
df['(excel) Total Num Images']  = ['']*len(pico_intanRaw_directories)
df['(excel) Notes']  = ['']*len(pico_intanRaw_directories)
df['Path: SpikeTimes']  = ['']*len(pico_intanRaw_directories)
df['Path: psth']  = ['']*len(pico_intanRaw_directories)
df['Path: h5']          = ['']*len(pico_intanRaw_directories)
df['Path: intanraw']    =  pico_intanRaw_directories

indices_spiketime = [index for index, name in enumerate(intan_names) if name in spike_names]
for ind in indices_spiketime:
    df.at[ind, 'Has SpikeTime'] = 1
    df.at[ind, 'Path: SpikeTimes'] = pico_spikeTime_directories[spike_names.index(intan_names[ind])]

indices_psth = [index for index, name in enumerate(intan_names) if name in psth_names]
for ind in indices_psth:
    df.at[ind, 'Has psth'] = 1
    df.at[ind, 'Path: psth'] = pico_psth_directories[psth_names.index(intan_names[ind])]



i = 0
for path, date in zip(pico_intanRaw_directories, dates):
    matching_h5files = find_files_with_pattern(os.path.join('/', *path.split('/')[0:10]), date)
    if len(matching_h5files) == 1:
        df.at[i, 'Path: h5'] = matching_h5files[0]
        df.at[i, 'Has h5'] = 1
    if len(matching_h5files) > 1:
        df.at[i, 'Path: h5'] = f'{len(matching_h5files)} files: {matching_h5files}'
        df.at[i, 'Has h5'] = 1
    i += 1

############### Load Excel File ###############################################
###############################################################################
path = os.path.dirname(cwd)+'/Pipeline Monkey Schedule New.xlsx'
data = pd.read_excel(path, sheet_name='pico')
new_header = data.iloc[0]  
data = data[1:]       
data.columns = new_header  
data = data.fillna('empty')  

def add_from_excel(df, data, index):
    df.at[ind, 'Has Excel']                 = 1
    df.at[ind, '(excel) Index']             = index+2
    df.at[ind, '(excel) Stimuli']           = data['Stimuli'][index]
    df.at[ind, '(excel) Num Reps']          = data['Reps'][index]
    df.at[ind, '(excel) Num Images']        = data['Images*/Videos'][index]
    df.at[ind, '(excel) Total Num Images']  = data['Total images/Videos'][index]
    df.at[ind, '(excel) Notes']  = data['Notes'][index]


for intan, ind in zip(pico_intanRaw_directories, range(len(pico_intanRaw_directories))):
    hasexcel = False

    ############### Get Recording Info from FileName ##############################
    ###############################################################################
    date = ((intan.split('/')[11].split('_')[-2]))
    time = ((intan.split('/')[11].split('_')[-1]))
    name = (intan.split('/')[7])
    found_item = False

    ############### Filter by Date in FileName and ExcelDate ##########################
    ###############################################################################
    for exl_date,index in zip(data['Date'], range(1,len(data))):
        exl_day, exl_month, exl_year = get_date(exl_date)
        
        if exl_year+exl_month+exl_day == date:

            ############### Filter by FileName and ExcelStimulus ##########################
            ###############################################################################
            list = [data['Stimuli'][index]]
            for i in range(1,10):
                try:
                    if data['Date'][index+i] == 'empty':
                        list.append(data['Stimuli'][index+i])
                        i+=1
                    else:
                        break
                except:break

            # check stim name
            for item, i in zip(list, range(len(list))): 
                if item.lower() in name:
                    add_from_excel(df, data, index+i)
                    found_item = True

            # check _ and -
            if found_item == False:
                for item, i in zip(list, range(len(list))):
                    name_new = name.lower().replace('-', '_').split('_')
                    item_new = item.lower().replace('-', '_').split('_') 

                    if set(item_new).intersection(set(name_new)) == set(item_new) or set(name_new).intersection(set(item_new)) == set(name_new):
                        add_from_excel(df, data, index+i)
                        found_item = True

            # manually correct rest
            if found_item == False:
                for item, i in zip(list, range(len(list))):
                    if name == 'monkeyvalence' and item.lower() in ['monkeyvalence2','monkeyvalence3', 'monkeyvalence4', \
                                                                    'monkeyvalence5', 'monkeyvalence6', 'monkeyvalence7', 'monkeyvalence8']:
                        add_from_excel(df, data, index+i)
                        found_item = True
                    if name == 'shapegen' and intan.split('/')[-1].split('_')[2] == 'static' and item.lower() == 'shapegen - static':
                        add_from_excel(df, data, index+i)
                        found_item = True
                    if name == 'shapegen' and intan.split('/')[-1].split('_')[2] == 'dynamic' and item.lower() == 'shapegen - dynamic':
                        add_from_excel(df, data, index+i)
                        found_item = True
                    if name == '1_shapes' and item.lower() == 'shapenet images':
                        add_from_excel(df, data, index+i)
                        found_item = True
                    if name == 'normalizers-HVM' and item.lower() == 'normalizers - hvm':
                        add_from_excel(df, data, index+i)
                        found_item = True
                    if name == 'facescrub-small' and item.lower() == 'faces_transformation':
                        add_from_excel(df, data, index+i)
                        found_item = True
                    if name == 'object_relations' and item.lower() == 'object_relationships':
                        add_from_excel(df, data, index+i)
                        found_item = True
                    if name == 'RF_Mapping_Yoon' and item in ['RF-Yoon Pico version', 'RFMapping_Yoon', 'RF-Yoon Pico version']:
                        add_from_excel(df, data, index+i)
                        found_item = True
                    if name == 'emogan' and item.lower() == 'emogen':
                        add_from_excel(df, data, index+i)
                        found_item = True
                    if name == 'IAPS' and intan.split('/')[-1].split('_')[2] == '200on' and item == 'IAPS - 200on':
                        add_from_excel(df, data, index+i)
                        found_item = True

    if found_item == False:
        print('No Excel: ', date, name, intan)
        
# display(df.tail(10))

############### Save New Excel File ###########################################
###############################################################################

excel_file_path = f'{os.path.dirname(cwd)}/{inventory_excel_file_name}'  
df.to_excel(excel_file_path, index=False) 


############### Add New Sheet to Excel File ###################################
###############################################################################

# Create a list to store the results
result_list = []

# Group the DataFrame by 'ImageSet' and iterate through the groups
for group, group_df in df.groupby('ImageSet'):
    image_set = group
    num_entries = len(group_df)
    num_has_excel = group_df['Has Excel'].astype(int).sum()
    num_has_spike_time = group_df['Has SpikeTime'].astype(int).sum()
    num_has_psth = group_df['Has psth'].astype(int).sum()
    num_has_h5 = group_df['Has h5'].astype(int).sum()
    
    # Append the results to the list
    result_list.append({
        'ImageSet': image_set,
        'Num Rec. Sess.': num_entries,
        'Num Has SpikeTime': num_has_spike_time,
        'Num Has psth': num_has_psth,
        'Num Has h5': num_has_h5,
        'Num Has Excel': num_has_excel,
        'BrainScore': '',
        'Comments': ''
    })

# Convert the list of dictionaries to a DataFrame
summary_df = pd.DataFrame(result_list)

# Optionally, set 'ImageSet' as the index
# summary_df.set_index('ImageSet', inplace=True)
summary_df = summary_df.sort_values(by='Num Rec. Sess.', ascending=False)
# Display the summary DataFrame
display(summary_df.head(5))


existing_excel_file = f'{os.path.dirname(cwd)}/{inventory_excel_file_name}' 

# Create a Pandas ExcelWriter object
with pd.ExcelWriter(existing_excel_file, engine='openpyxl', mode='a') as writer:
    # Write your DataFrame to a new sheet
    summary_df.to_excel(writer, sheet_name='Sheet2', index=False)  # 'Sheet2' is the name of the new sheet


No Excel:  220719 normalizers /braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/intanraw/pico_normalizers_220719_140828
No Excel:  230922 normalizers /braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/intanraw/pico_normalizers_230922_134247
No Excel:  230929 normalizers /braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/intanraw/pico_normalizers_230929_095756
No Excel:  230106 normalizers /braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/intanraw/pico_normalizers_230106_162941
No Excel:  230920 normalizers /braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/intanraw/pico_normalizers_230920_111320
No Excel:  230925 normalizers /braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/intanraw/pico_normalizers_230925_111712
No Excel:  230927 normalizers /braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/intanraw/pico_normalizers_230927_100801

,ImageSet,Num Rec. Sess.,Num Has SpikeTime,Num Has psth,Num Has h5,Num Has Excel,BrainScore,Comments
53,normalizers,168,127,122,93,160,,
54,normalizers-HVM,34,20,20,20,31,,
36,mayo,14,14,14,10,14,,
50,muri1320,12,12,12,12,12,,
17,domain-transfer-2023,12,12,12,12,12,,


### Step 4: Create/update inventory

In [14]:
############### Create Directories ############################################
###############################################################################

df = pd.read_excel( f'{os.path.dirname(cwd)}/{inventory_excel_file_name}'  )
SubjectName = 'pico'
storage_dir = '/braintree/home/aliya277/inventory_new'
storage_old = '/braintree/home/aliya277/inventory'

for index, DataFrame in df.iterrows():
        
    if DataFrame['Has SpikeTime'] == 1:
        date = f"20{DataFrame['date']}"
        if len(str(DataFrame['time'])) != 6: time = f"0{DataFrame['time']}"
        else: time = str(DataFrame['time'])
        
        if DataFrame['ImageSet'] == 'normalizers':
            directory = f'norm_FOSS.sub_pico.{date}_{time}.proc'
        elif DataFrame['ImageSet'] == 'normalizers-HVM':
            directory = f'norm_HVM.sub_pico.{date}_{time}.proc'
        else: 
            directory = f"exp_{DataFrame['ImageSet']}.sub_pico.{date}_{time}.proc"

        imagesetdir = os.path.join(storage_dir, ".".join(directory.split(".")[0:1]))
        subjectdir  = os.path.join(storage_dir, imagesetdir, ".".join(directory.split(".")[0:2]))
        subjectdir_date  = os.path.join(subjectdir, ".".join(directory.split(".")[0:2])+'.'+date)
        print(DataFrame['ImageSet'])
        try: os.mkdir(imagesetdir)
        except: pass
        try: os.mkdir(subjectdir)
        except: pass
        try: os.mkdir(subjectdir_date, directory)
        except: pass
            
        if not os.path.isdir(os.path.join(subjectdir_date, directory, 'SpikeTimes')):
            print(f'Copying SpikeTimes for {directory}')
            shutil.copytree(DataFrame['Path: SpikeTimes'], os.path.join(subjectdir_date, directory, 'SpikeTimes'))
            
        if not os.path.isdir(os.path.join(subjectdir_date, directory, 'psth')) and DataFrame['Has psth'] == 1: 
            print(f'Copying psth for {directory}')
            os.mkdir(os.path.join(subjectdir_date, directory, 'psth'))
            psth_path = DataFrame['Path: psth']
            n_channels = len(os.listdir(DataFrame['Path: SpikeTimes']))
            path_intanproc = os.path.join('/', *psth_path.split('/')[:-1])
            # ------------------------------------------------------------------------------ 
            # copy psth from intanproc path to inventory
            # ------------------------------------------------------------------------------ 
            combined = False
            for ele in os.listdir(path_intanproc):
                if ele.endswith("_psth.mat"): 
                    combined = True
                    shutil.copy2(path_intanproc+'/'+ele, os.path.join(subjectdir_date, directory, 'psth', ele))
            # ------------------------------------------------------------------------------ 
            # combine channels and save in inventory
            # ------------------------------------------------------------------------------ 
            if not combined: 
                n_channels = len(os.listdir(path_intanproc+'/psth'))
                combine_channels(psth_path, os.path.join(subjectdir_date, directory, 'psth'), n_channels)


normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
normalizers
norm

In [5]:
############### Add Sarah's BrainScore Column #################################
###############################################################################

df = pd.read_excel( f'{os.path.dirname(cwd)}/{inventory_excel_file_name}' , sheet_name='Sheet2').sort_values(by='ImageSet')
df_sarah = pd.read_excel( f'{os.path.dirname(cwd)}/pico_inventory_sarah.xlsx', sheet_name='Sheet2').sort_values(by='ImageSet')

for ind, row in df.iterrows():
    if not row['ImageSet'] in df_sarah['ImageSet'].tolist(): print(f"New ImageSets: {row['ImageSet']}")
    index = np.where(df_sarah["ImageSet"].to_numpy() == row['ImageSet'])[0]
    if len(df_sarah.iloc[index]['BrainScore'].tolist()) > 0: df.at[ind,"BrainScore"] = df_sarah.iloc[index]['BrainScore'].tolist()[0]
    if len(df_sarah.iloc[index]['Comments'].tolist()) > 0:   df.at[ind,"Comments"] = df_sarah.iloc[index]['Comments'].tolist()[0]

# Update Sheet 2
xls = pd.ExcelFile(f'{os.path.dirname(cwd)}/{inventory_excel_file_name}')
sheets = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}

sheets['Sheet2'] = df  

with pd.ExcelWriter(f'{os.path.dirname(cwd)}/{inventory_excel_file_name}', engine='openpyxl', mode='w') as writer:
    for sheet_name, sheet_df in sheets.items():
        sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)


In [ ]:
# ------------------------------------------------------------------------------ 
# Delete all faulty normalizer files.
# ------------------------------------------------------------------------------ 

experiment_file_paths = glob.glob(os.path.join(root_dir, '*', '*', '*', '*'))

from scipy.io import loadmat
import shutil

for experiment_path in experiment_file_paths: 
    experiment_name =  "_".join(os.path.basename(experiment_path).split('.')[0].split('_')[1:])
    for file in os.listdir(experiment_path):
        if file == 'psth': 
            path = os.path.join(experiment_path, 'psth', os.listdir(os.path.join(experiment_path, file))[0])
            mat = loadmat(path)
            if mat['psth'].shape[-1] != 288 and mat['psth'].shape[-1] != 192:
                print(mat['psth'].shape)
                shutil.rmtree(os.path.join(experiment_path, 'psth'))
            del mat


### Step 5: Create inventory from path (only for new data not from sarah's pico data)

In [26]:
############### Create Directories ############################################
###############################################################################

storage_dir = '/braintree/home/aliya277/inventory_new'

experiment_path = '/braintree/home/aliya277/sachis_data/bold5000/intanproc'
# experiment_path = '/braintree/home/aliya277/sachis_data/nat300/intanproc'
# experiment_path = '/braintree/home/aliya277/sachis_data/hvm/intanproc'

experiment_names = os.listdir(experiment_path)

for experiment in experiment_names:
    if not experiment.startswith('.'):
        subj        = experiment.split('_')[0]
        imageset    = experiment.split('_')[1]
        date        = experiment.split('_')[-2]
        time        = experiment.split('_')[-1]

        if len(date) != 8: date = f'20{date}'

        # ------------------------------------------------------------------------------ 
        # Create directories in inventory.
        # ------------------------------------------------------------------------------ 

        directory = f"exp_{imageset}.sub_{subj}.{date}_{time}.proc"

        imagesetdir = os.path.join(storage_dir, ".".join(directory.split(".")[0:1]))
        subjectdir  = os.path.join(storage_dir, imagesetdir, ".".join(directory.split(".")[0:2]))
        subjectdir_date  = os.path.join(subjectdir, ".".join(directory.split(".")[0:2])+'.'+date)
        try: os.mkdir(imagesetdir)
        except: pass
        try: os.mkdir(subjectdir)
        except: pass
        try: os.mkdir(subjectdir_date)
        except: pass
        try: os.mkdir(os.path.join(subjectdir_date, directory))
        except: pass
        

        # ------------------------------------------------------------------------------ 
        # Copy SpikeTimes and PSTH in inventory.
        # ------------------------------------------------------------------------------ 

        path_intanproc = os.path.join(experiment_path, experiment)
        if os.path.isdir(path_intanproc):
            for ele in os.listdir(path_intanproc):
                if ele.endswith("_psth.normalized.mat") and not ele.startswith(".") and not os.path.isdir(os.path.join(subjectdir_date, directory, 'psth_normalized')): 
                    print(f'Copying normalized psth for {directory}')
                    try:  os.mkdir(os.path.join(subjectdir_date, directory, 'psth_normalized'))
                    except: pass
                    shutil.copy2(path_intanproc+'/'+ele, os.path.join(subjectdir_date, directory, 'psth_normalized', ele))

            if not os.path.isdir(os.path.join(subjectdir_date, directory, 'SpikeTimes')):
                print(f'Copying SpikeTimes for {directory}')
                shutil.copytree(os.path.join(path_intanproc,'spikeTime'), os.path.join(subjectdir_date, directory, 'SpikeTimes'))


            if not os.path.isdir(os.path.join(subjectdir_date, directory, 'psth')):
                print(f'Copying psth for {directory}')
                psth_path = os.path.join(path_intanproc,'psth')
                n_channels = len(os.listdir(psth_path))
                os.mkdir(os.path.join(subjectdir_date, directory, 'psth'))
                # print(n_channels)
                combine_channels(psth_path, os.path.join(subjectdir_date, directory, 'psth'), n_channels)
            


Copying SpikeTimes for exp_bold5000.sub_solo.20190220_143521.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190220_160047.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190221_095435.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190222_120151.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190225_111537.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190226_094812.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190227_122653.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190304_120611.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190305_112242.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190306_122612.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190307_132706.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190308_120834.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190311_113347.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190312_130545.proc
Copying SpikeTimes for exp_bold5000.sub_solo.20190317_115104.proc
Copying Sp